In [ ]:
import pandas as pd

In [ ]:
import networkx as nx
from BeyondBlooms2024.config import name_dict
from BeyondBlooms2024.config_file import (ABUNDANCES_FILE, CCMN_CON_MAP_PATH, CON_LOUVAIN_META_PATH,CON_LOUVAIN_NETWORK_PATH, ENRICH
,NUM_PERMUTATIONS, NUM_SAMPLES, NUM_CORES, METADATA_FILE, PRUNED_PVAL_CCMN_PATH,PVAL_CCMN_PATH,ENRICHED_META_PATH, RANDOM_PVAL_CCMN_PATH)

In [ ]:
df_ccm = pd.read_csv(PRUNED_PVAL_CCMN_PATH,sep=";")
df_ccm.dropna(inplace=True)
print(df_ccm.shape)
df_ccm.head()

In [ ]:
df_ccm = df_ccm[~(df_ccm['from_clu'].isin([-1]) & df_ccm['to_clu'].isin([-1]))]
print(df_ccm.shape)

In [ ]:
edges_count = len(df_ccm)
nodes_count = len(pd.concat([df_ccm['from'], df_ccm['to']]).unique())

print("Number of edges (CCM):", edges_count)
print("Number of nodes: (CCM)", nodes_count)

In [ ]:
G = nx.from_pandas_edgelist(df_ccm, 'from', 'to', ['corr', 'p-value'])
df = df_ccm

In [ ]:
#df['from_clu'] = df["from"].apply(lambda x: apply_dict(x, meta_dict))
#df['to_clu'] = df["to"].apply(lambda x: apply_dict(x, meta_dict))
#df['from_clu_'] = df['from_clu']
#df['to_clu_'] = df['to_clu']
df['from_clu'] = df['from_clu'].apply(lambda x: name_dict[str(x)])
df['to_clu'] = df['to_clu'].apply(lambda x: name_dict[str(x)])
df = df[~(df['from_clu'].isin(["RAUS"]))]
df = df[~(df['to_clu'].isin(["RAUS"]))]
#print(df['to_clu'])
#df

In [ ]:
name_dict

In [ ]:
def create_combined_column(row):
    if row['from_clu'] == row['to_clu']:
        return row['from_clu']
    else:
        return f"{row['from_clu']}->{row['to_clu']}"

def create_combined_column_(row):
    if row['from_clu'] == row['to_clu']:
        return row['from_clu']
    else:
        return f"{row['from_clu']}->"

def create_combined_column__(row):
    if row['from_clu'] == row['to_clu']:
        return row['from_clu']
    else:
        return f"->{row['to_clu']}"

df['egdes_between_clu'] = df.apply(create_combined_column, axis=1)
df['egdes_from_cluster'] = df.apply(create_combined_column_, axis=1)
df['egdes_cluster'] = df.apply(create_combined_column__, axis=1)
print(df)

In [ ]:
print(df['egdes_between_clu'].value_counts())

In [ ]:
df.columns

In [ ]:
# print(df['egdes_between_clu'].value_counts().index)
import numpy as np
from scipy import stats
def mode(x):
    return stats.mode(x)[0]
#df['egdes_between_clu'] =df['egdes_between_clu'].astype(int)
between = df.groupby(['egdes_between_clu']).aggregate({"corr": "mean", "from_clu": "first", "to_clu": "first",}).reset_index()
between_2 = between#[~between["egdes_between_clu"].isin(["01-F","02-F","03-L","04-L","05-L","06-M","07-M","08-M","09-H","10-H"])] #ToDo: names
print(between_2.shape)
#between_2["from_clu_"] =between_2["from_clu_"].astype(int)
#between_2["to_clu_"] =between_2["to_clu_"].astype(int)
print(between_2)
between_3 = between_2[["egdes_between_clu","from_clu", "to_clu", "corr"]]
#between_3["from_clu_"] =between_3["from_clu"].apply(lambda x: name_dict[str(x)])
#between_3["to_clu_"] =between_3["to_clu"].apply(lambda x: name_dict[str(x)])
between_3.to_csv("tables/Sup_Figure_3c_network_table.csv", sep=";")
between_3

In [ ]:
print(between_3)

In [ ]:
between_4 = between[between["egdes_between_clu"]!="RAUS"]
#print(between_4.head())
df_taxa_3 = pd.read_csv(ENRICH, sep =',', engine="python")
df_taxa_3 = df_taxa_3[~df_taxa_3["cluster_names"].isin(["RAUS"])]
df_taxa_3.head()

In [ ]:
def mode_first(x):
    modes = x.mode()
    if isinstance(modes, pd.Series):
        return modes.iloc[0]  # Return the first mode if it's a single value
    elif isinstance(modes, pd.DataFrame):
        return modes.iloc[0, 0]  # Return the first value of the first mode if it's a list
    else:
        return None
    

In [ ]:
aad = df_taxa_3[["LouvainLabelD","MaxMonth","MaxMonthColor","louvain_label_color"]].groupby(['LouvainLabelD']).aggregate(mode_first).reset_index()
#[["LouvainLabelD", "MaxMonth"]]
aad["Cluster"]=aad['LouvainLabelD']
aad.set_index("LouvainLabelD",inplace=True)
aad = aad[["Cluster","MaxMonth","MaxMonthColor","louvain_label_color"]]
aad["Cluster_"]= aad["Cluster"].apply(lambda x: name_dict[str(int(x))])
print(aad)
aad.to_csv("tables/Sup_Figure_3c_table_meta.csv", sep=";")

In [ ]:
def month_to_season(month_value:int):

    month_value = int(np.round(month_value))
    if month_value in [9,10,11]:
        ret = "F"
    elif month_value in [12,1,2]:
        ret = "LL"
    elif month_value in [3,4,5]:
        ret = "M"
    else:
        ret = "HL"
    return ret

In [ ]:
#df_meta.value_counts("LouvainLabelD")

In [ ]:
df_meta = pd.read_csv(ENRICH,sep=",")
meta_dict_ =df_meta[["Nodes","cluster_names"]]
meta_dict_ =meta_dict_.set_index("Nodes")
meta_dict = meta_dict_.to_dict()["cluster_names"]
print(meta_dict)

In [ ]:
def apply_(x,ob):
    try:
        ret =ob[x]
    except:
        ret= np.nan
    return ret

In [ ]:
df_table = pd.read_csv(CON_LOUVAIN_NETWORK_PATH,sep=";")
df_table['from_clu'] = df_table["from"].map(meta_dict)
df_table['to_clu'] = df_table["to"].map(meta_dict)
df_table['egdes_between_clu'] = df_table.apply(create_combined_column, axis=1)
#df_table.value_counts('egdes_between_clu')

aad["Season"] = aad["MaxMonth"].apply(month_to_season)
#aad["Season T"] = aad["MaxMonthTotal"].apply(month_to_season)
aad2 =aad[["Cluster", "MaxMonth", "Season"]]
aad2["NumberOfAsv"]= aad2["Cluster"].apply(lambda x: df_meta.value_counts("LouvainLabelD")[int(x)])
#aad2["Connected"] = aad2["NumberOfAsv"].apply(lambda x: True if x > 10 else False)
aad2.sort_values(by="Season")
aad2["NumberOfEdges CCM"] = aad2["Cluster"].apply(lambda x: apply_(df.value_counts("egdes_between_clu"),int(np.round(x))))
aad2["NumberOfEdges CON"] = aad2["Cluster"].apply(lambda x: apply_(df_table.value_counts('egdes_between_clu'),int(np.round(x))))
aad2 =aad2[["Cluster", "MaxMonth", "Season","NumberOfAsv","NumberOfEdges CCM","NumberOfEdges CON"]]
#aad2 = aad2[aad2["Cluster"].isin([0,1,2,4,6,7,8,9,10,11])]
aad2["Cluster"] =aad2["Cluster"].apply(lambda x: name_dict[str(int(x))])
aad2.sort_values("Cluster", inplace=True)
aad2.to_csv("tables/Sup_Figure_Unknown_Cluster_overview.csv", sep=";")
aad2

In [ ]:
#df_meta.value_counts("LouvainLabelD")

In [ ]:
df["number_of_nodes_total"] = nodes_count
df["number_of_edges_total"] = edges_count

In [ ]:
df.to_csv("tables/Sup_ADD_CC_Network_Analyis_overview_0818.csv", sep=";")

In [ ]:
print(df.head())